In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sentiment Analysis using AutoML Natural Language and Vertex AI

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Table of contents
* [Overview](#section-1)
* [Objective](#section-2)
* [Dataset](#section-3)
* [Costs](#section-4)
* [Load the data](#section-5)
* [Prepare the training data](#section-6)
* [Create a dataset in Vertex AI](#section-7)
* [Train the model using Vertex AI](#section-8)
* [Deploy the model to the endpoint](#section-9)
* [Prediction](#section-10)
* [Review visualization](#section-11)
* [Clean up](#section-12)

## Overview
<a name="section-1"></a>

This notebook demonstrates how to perform sentiment analysis on a Stanford movie reviews dataset using AutoML Natural Language and how to deploy the sentiment analysis model on Vertex AI to get predictions. 

*Note: This notebook file was developed to run on a [Vertex AI Workbench managed notebooks](https://console.cloud.google.com/vertex-ai/workbench/list/managed) instance using the Python (Local) kernel. Some components of this notebook may not work in other notebook environments.*

Learn more about [Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) and [AutoML Text](https://cloud.google.com/vertex-ai/docs/tutorials/text-classification-automl/training).

### Objective
<a name="section-2"></a>

In this tutorial, you learn how to train and deploy an AutoML sentiment analysis model, and make predictions.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Dataset`
- `Vertex AI Model`
- `AutoML training`
- `Vertex AI Prediction`

The steps performed are:

- Loading the required data. 
- Preprocessing the data.
- Selecting the required data for the model.
- Loading the dataset into Vertex AI managed datasets.
- Training a sentiment model using AutoML Text training.
- Evaluating the model.
- Deploying the model on Vertex AI.
- Getting predictions.
- Clean up.

### Dataset
<a name="section-3"></a>

The dataset used in this notebook is a part of the [Stanford Sentiment Treebank dataset](https://nlp.stanford.edu/sentiment/), which consists of phrases from movie reviews and their corresponding sentiment scores.

### Costs
<a name="section-4"></a>

This tutorial uses the following billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Select <b>Python</b> kernel while running this notebook on Vertex AI's managed instances and ensure that the following libraries are installed in the environment where this notebook is being run.

- wordcloud
- Pandas 

Along with the above libraries, the following google-cloud libraries are also used in this notebook.

- google.cloud.aiplatform
- google.cloud.storage

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    
! pip3 install --upgrade wordcloud {USER_FLAG} -q

! pip3 install --upgrade google-cloud-aiplatform \
                        fsspec \
                        gcsfs {USER_FLAG} -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI and Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench notebooks**, your environment is already authenticated.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you create a Vertex AI's managed dataset resource using the Cloud SDK, you specify the Cloud Storage bucket where the data is stored. Vertex AI manages the data from the provided location. In this tutorial, Vertex AI creates a managed text dataset from the data provided in a Cloud Storage bucket. On the created managed dataset, you can then train an AutoML Natural Language model and deploy it to an endpoint for serving online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents.

In [ ]:
! gsutil ls -al $BUCKET_URI

### Import libraries

In [1]:
import os
from typing import List, Optional, Union

import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import aiplatform, storage
from wordcloud import STOPWORDS, WordCloud

## Load the data 
<a name="section-5"></a>

Load the phrases and scores of the dataset from the Cloud Storage sources.

In [ ]:
phrases = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/sentiment_analysis/stanford_sentiment_treebank/sentiment_phrases.txt",
    sep="|",
)
phrases.columns = ["text", "phrase ids"]
scores = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/sentiment_analysis/stanford_sentiment_treebank/sentiment_labels.txt",
    sep="|",
)
df = phrases.merge(scores, how="left", on="phrase ids")
print(df.head(5))

In [ ]:
print(max(df["sentiment values"]), min(df["sentiment values"]))

The data itself doesn't contain any feature names and thus needs its columns to be renamed. The dataframe **phrases** contains all phrases and their IDs separated by `|` whereas **scores** contains all phrase ids and the corresponding sentiment scores separated by a `|`.

### Create labels 

The label scores are bucketed into four classes by mapping them into the following ranges:

[0, 0.25], (0.25, 0.5], (0.5, 0.75],(0.75, 1.0]

In [ ]:
# Define the class labels
VERYNEGATIVE = 0
NEGATIVE = 1
POSITIVE = 2
VERYPOSITIVE = 3

In [ ]:
bins = [0, 0.25, 0.5, 0.75, 1]
labels = [VERYNEGATIVE, NEGATIVE, POSITIVE, VERYPOSITIVE]
df["label"] = pd.cut(df["sentiment values"], bins=bins, labels=labels)
print(df.head())

## Prepare the training data
<a name="section-6"></a>

To train a sentiment analysis model using AutoML Natural Language, you provide samples of the type of content you want to train labeled with a value indicating how the sentiment varies.

The sentiment label is encoded as integer ranging from 0 (relatively negative) to a maximum value of your choice (positive). For example, if you want to identify whether the sentiment is negative, positive, or neutral, you would label the training data with sentiment scores of 0 (negative), 1 (neutral), and 2 (positive). If you want to capture more granularity with five levels of sentiment, you can still label documents with the most negative sentiment as 0 and use 4 for the most positive sentiment. In such case, the maximum sentiment score (sentiment_max) for the dataset would be 4.

For training, you select a subset of the orginal data that consists of extreme positive and negative samples. Here the maximum sentiment would be 1. In the **ml_use** column, you provide if the instance belongs to TRAIN, VALIDATION or TEST sample or let the Vertex AI randomly assign. 

Each line or instance in the CSV file refers to a single document. The following example shows the general format of a valid CSV file:

**[ml_use]**,**gcs_file_uri**|**"inline_text"**,**sentiment**,**sentimentMax**

For more information visit the official [documentation](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#sentiment-analysis) on preparing the sentiment analysis data.


### Select a subset of the data

In [ ]:
subset_data = df[df["label"].isin([VERYNEGATIVE, VERYPOSITIVE])].reset_index(drop=True)
subset_data.head()

In [ ]:
subset_data["label"] = subset_data["label"].apply(lambda x: 1 if x == 3 else 0)
subset_data["ml_use"] = ""
subset_data["sentimentMax"] = 1
subset_data = subset_data[["ml_use", "text", "label", "sentimentMax"]]
print(subset_data.head())

### Create a csv

In [ ]:
FILE_NAME = "sentiment_data.csv"
subset_data.to_csv(FILE_NAME, index=False)
# Upload the saved model file to Cloud Storage
BLOB_PATH = "sentiment_analysis/"
BLOB_NAME = os.path.join(BLOB_PATH, FILE_NAME)
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

## Create a dataset in Vertex AI
<a name="section-7"></a>

The following code uses the Vertex AI SDK for Python to both create a dataset and import data. 

In [ ]:
def import_data_text_sentiment_analysis(
    project: str,
    location: str,
    display_name: str,
    src_uris: Union[str, List[str]],
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    ds = aiplatform.TextDataset.create(
        display_name=display_name,
        gcs_source=src_uris,
        import_schema_uri=aiplatform.schema.dataset.ioformat.text.sentiment,
        sync=sync,
    )

    print(ds.display_name)
    print(ds.resource_name)
    return ds

Set a display name for the managed dataset. 

In [ ]:
DATASET_NAME = "[your-dataset-display-name]"  # @param {type:"string"}

In [ ]:
if DATASET_NAME == "" or DATASET_NAME == "[your-dataset-display-name]":
    DATASET_NAME = "sentimentanalysis"

# Create a Vertex AI managed dataset resource
src_uris = [f"{BUCKET_URI}/sentiment_analysis/sentiment_data.csv"]
dataset = import_data_text_sentiment_analysis(
    PROJECT_ID, REGION, DATASET_NAME, src_uris
)

## Train the model using Vertex AI
<a name="section-8"></a>

The following code uses the Vertex AI SDK for Python to train the model on the above created dataset. You can get the dataset id from the Dataset section of Vertex AI in the console or from the resource name in the dataset object created above. You can specify how the training data is split between the training, validation, and test sets by setting the fraction_split variables.

**Note**: The model training job can take more than 120 min. to finish.

In [ ]:
def create_training_pipeline_text_sentiment_analysis(
    project: str,
    location: str,
    display_name: str,
    dataset_id: str,
    model_display_name: Optional[str] = None,
    sentiment_max: int = 10,
    training_fraction_split: float = 0.8,
    validation_fraction_split: float = 0.1,
    test_fraction_split: float = 0.1,
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    job = aiplatform.AutoMLTextTrainingJob(
        display_name=display_name,
        prediction_type="sentiment",
        sentiment_max=sentiment_max,
    )

    text_dataset = aiplatform.TextDataset(dataset_id)

    model = job.run(
        dataset=text_dataset,
        model_display_name=model_display_name,
        training_fraction_split=training_fraction_split,
        validation_fraction_split=validation_fraction_split,
        test_fraction_split=test_fraction_split,
        sync=sync,
    )

    print(model.display_name)
    print(model.resource_name)
    print(model.uri)
    return model

Set a display name for the AutoML training job.

In [ ]:
JOB_NAME = "[your-training-job-display-name]"  # @param {type:"string"}

In [ ]:
if JOB_NAME == "" or JOB_NAME == "[your-training-job-display-name]":
    JOB_NAME = "sentimentanalysis_job"

# Using the created Text dataset, create and run the training job
dataset_id = dataset.resource_name.split("/")[-1]
print(dataset_id)
model = create_training_pipeline_text_sentiment_analysis(
    PROJECT_ID, REGION, JOB_NAME, dataset_id, sentiment_max=1
)

## Deploy the model to the endpoint
<a name="section-9"></a>


### Create endpoint
Create a Vertex AI endpoint to deploy the model.

In [ ]:
def create_endpoint(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

Set a display name for the endpoint.

In [ ]:
ENDPOINT_NAME = "[your-endpoint-name]"  # @param {type:"string"}

In [ ]:
if ENDPOINT_NAME == "[your-endpoint-name]":
    ENDPOINT_NAME = "sentiment-analysis-endpoint"

# Create Endpoint
endpoint = create_endpoint(PROJECT_ID, ENDPOINT_NAME, REGION)

### Deploy the model

The following code uses the Vertex AI SDK for Python to deploy the model to a endpoint.

In [ ]:
model.deploy(endpoint=endpoint)

print(model.display_name)
print(model.resource_name)

## Prediction
<a name="section-10"></a>

After deploying the model to an endpoint use the Vertex AI SDK to request an online prediction. Filter the data that you haven't used for the training and pick longer reviews to test the model.

In [ ]:
def predict_text_sentiment_analysis_sample(endpoint, content):
    print(content)
    response = endpoint.predict(instances=[{"content": content}], parameters={})

    for prediction_ in response.predictions:
        print(prediction_)

In [ ]:
test_data_pos = df[df["label"].isin([POSITIVE])].reset_index(drop=True)
test_data_neg = df[df["label"].isin([NEGATIVE])].reset_index(drop=True)

test_data_neg = test_data_neg.text.values
test_data_neg = [i for i in test_data_neg if len(i) > 200]
random.shuffle(test_data_neg)

In [ ]:
test_data_pos = test_data_pos.text.values
test_data_pos = [i for i in test_data_pos if len(i) > 200]
random.shuffle(test_data_pos)

Here is the prediction results on the positive samples. The model did a good job on predicting positive sentiment for positive reviews. The first and last review predictions are false negatives. 

In [ ]:
for review in test_data_pos[0:10]:
    predict_text_sentiment_analysis_sample(endpoint, review)

Here is the prediction results on the negative reviews. Out of 10 reviews below 7 negative reviews are correctly predicted with negative sentiment

In [ ]:
for review in test_data_neg[0:10]:
    predict_text_sentiment_analysis_sample(endpoint, review)

## Review visualization
<a name="section-11"></a>

Visualize the positive and negative reviews in the data.

In [ ]:
data_pos = df[df["label"].isin([VERYPOSITIVE])].reset_index(drop=True)
data_neg = df[df["label"].isin([VERYNEGATIVE])].reset_index(drop=True)

data_neg = data_neg.text.values

In [ ]:
data_pos = data_pos.text.values

Create the word cloud by removing the common words to highlight the words representing positive and negative samples.

In [ ]:
# Python program to generate WordCloud
def plot_word_cloud(data, common_words):
    comment_words = ""
    stopwords = set(STOPWORDS)
    for val in data:
        tokens = val.split()
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
            for each in common_words:
                if each in tokens[i]:
                    tokens[i] = ""
                    break

        comment_words += " ".join(tokens) + " "

    wordcloud = WordCloud(
        width=800,
        height=800,
        background_color="white",
        stopwords=stopwords,
        min_font_size=10,
    ).generate(comment_words)

    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)

    plt.show()

Plot a word cloud of negative reviews.

In [ ]:
plot_word_cloud(
    data_neg,
    [
        "movie",
        "film",
        "story",
        "audience",
        "director",
        "watch",
        "seem",
        "world",
        "one",
        "make",
        "way",
        "character",
        "much",
        "time",
        "even",
        "take",
        "s",
        "n't",
        "will",
        "may",
        "re",
        "plot",
        "good",
        "comedy",
        "made",
    ],
)

Plot a word cloud of positive reviews.

In [ ]:
plot_word_cloud(
    data_pos,
    [
        "movie",
        "film",
        "story",
        "audience",
        "director",
        "watch",
        "seem",
        "world",
        "one",
        "make",
        "way",
        "character",
        "much",
        "time",
        "even",
        "take",
        "s",
        "n't",
        "will",
        "may",
        "re",
        "plot",
        "made",
    ],
)

## Clean up
<a name="section-12"></a>

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy the model from the endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the dataset
dataset.delete()

Set `delete_bucket` to **True** to delete the Cloud Storage bucket.

In [ ]:
delete_bucket = False
# Delete the Cloud storge bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI